<a href="https://colab.research.google.com/github/NicoEssi/Data_Science_Portfolio/blob/master/Stackoverflow_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Setup

In [0]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
# Downloading the Stack Overflow Survey Results for 2019
!wget --no-check-certificate \
    "https://drive.google.com/uc?authuser=0&id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV&export=download" \
    -O "/tmp/soi_2019.zip"
zip_ref = zipfile.ZipFile("/tmp/soi_2019.zip", 'r')
zip_ref.extractall("/tmp/soi_2019")
zip_ref.close()

--2019-09-11 19:25:01--  https://drive.google.com/uc?authuser=0&id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV&export=download
Resolving drive.google.com (drive.google.com)... 74.125.195.101, 74.125.195.139, 74.125.195.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.195.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/uc?id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV&export=download [following]
--2019-09-11 19:25:01--  https://drive.google.com/uc?id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV&export=download
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/l36ap7cnpi8errreq9hcmeehjvdfhnju/1568224800000/06716978924947585995/*/1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV?e=download [following]
--2019-09-11 19:25:03--  https://doc-14-4c-docs.googleusercontent.com/docs/securesc

## Custom Functions

In [0]:
# Encode single label
def encode(data):
    return data.str.get_dummies()

# Encode multiple labels
def encodem(data):
    return data.str.get_dummies(sep=";")

In [0]:
# Parse all types within a data into a list
def parse_types(data):
    types_temp = []
    for i in range( len(data)):
        types_temp.append([parsed.strip() for parsed in str(data.iloc[i]).split(';')])
    
    types = []
    for i in types_temp:
        for j in i:
            if j not in types:
                types.append(j)
                
    return types

# Parse datarow of all text into a list
def parse(data):
    return [parsed.strip() for parsed in str(data).split(';')]

# Check whether text exists in data
def check(text, data):
    if text in parse(data):
        return 1
    else:
        return 0

# Count occurences of text in data
def countd(text, data):
    counted = []
    
    for i in data:
        counted.append(check(text, i))
    
    return sum(counted)

In [0]:
# Check salary for specific category
def check_average_salary(category, column, data):
    salaries = []
    for i in range( len(data) ):
        if category in parse(data[column].iloc[i]):
            salaries.append(data.CompTotal.iloc[i])
    return sum(salaries)/len(salaries)


# Check salary for specific column
def check_average_salary_column(column, data):
    for i in parse_types(data[column]):
        print(str(i) + ": " + str(check_average_salary(i, column, data)))

In [0]:
# currency rates as of 09/09/2019
"""
EUR = 1.10
CHF = 1.00
SEK = 0.10
NOK = 0.11
DKK = 0.15
USD = 1.00
GBP = 1.23
INR = 0.014
NZD = 0.64
AUD = 0.69
CAD = 0.76
"""

currencies = {"EUR" : 1.10
             ,"CHF" : 1.00
             ,"SEK" : 0.10
             ,"NOK" : 0.11
             ,"DKK" : 0.15
             ,"USD" : 1.00
             ,"GBP" : 1.23
             ,"INR" : 0.014
             ,"NZD" : 0.64
             ,"AUD" : 0.69
             ,"CAD" : 0.76}

# Convert currency into USD
def currency_usd(data):
    if data in (currencies):
        return currencies[data]
    else:
        return 0

# Annualize total compensation
def currency_annualize(data):
    if data == 'Weekly':
        return 52
    elif data == 'Monthly':
        return 12
    elif data == 'Yearly':
        return 1

# 1. Business- & Data Understanding

Read the data and initial exploration.

In [0]:
data = pd.read_csv("/tmp/soi_2019/survey_results_public.csv")
schema = pd.read_csv("/tmp/soi_2019/survey_results_schema.csv")

In [0]:
#for i in range(len(schema)):
#    print(schema.iloc[i].Column + " : " + schema.iloc[i].QuestionText)

Clean data of rows with missing salary information.

In [21]:
print("US Count: " + str(np.sum(data.Country == "United States")))
print("UK Count: " + str(np.sum(data.Country == "United Kingdom")))
print("EU Count: " + str(np.sum(data.Country == "France")
                        + np.sum(data.Country == "Germany")
                        + np.sum(data.Country == "Sweden")
                        + np.sum(data.Country == "Denmark")
                        + np.sum(data.Country == "Finland")
                        + np.sum(data.Country == "Ireland")
                        + np.sum(data.Country == "Netherlands")
                        + np.sum(data.Country == "Austria")
                        + np.sum(data.Country == "Belgium")
                        + np.sum(data.Country == "Switzerland")))

US Count: 20949
UK Count: 5737
EU Count: 15591


In [22]:
print("Before: " + str(len(data.CompTotal)))

data = data[pd.notnull(data['CompFreq'])]
data = data[pd.notnull(data['CompTotal'])].reset_index(drop = True)

print("After: " + str(len(data.CompTotal)))

Before: 88883
After: 55827


In [23]:
print("US Count: " + str(np.sum(data.Country == "United States")))
print("UK Count: " + str(np.sum(data.Country == "United Kingdom")))
print("EU Count: " + str(np.sum(data.Country == "France")
                        + np.sum(data.Country == "Germany")
                        + np.sum(data.Country == "Sweden")
                        + np.sum(data.Country == "Denmark")
                        + np.sum(data.Country == "Finland")
                        + np.sum(data.Country == "Ireland")
                        + np.sum(data.Country == "Netherlands")
                        + np.sum(data.Country == "Austria")
                        + np.sum(data.Country == "Belgium")
                        + np.sum(data.Country == "Switzerland")
                        + np.sum(data.Country == "Norway")))

US Count: 14981
UK Count: 4036
EU Count: 10744


## We are interested in identifying the profiles of the top earners in each region; United States, United Kingdom, and Europe. Thereafter, we'll build a model to predict salary.

### Why do we conduct our inquiry on two countries, yet one is of a larger region consisting of numerous countries (Europe)?
Professionals residing within the European Union have liberties granted to them by the Schengen agreement, which enables free movement of labor force and thus professionals can freely work wherever they wish.

### Why are Norway and Switzerland included in the European dataset?
While they are not part of the European Union, they have signed agreements in association with the Schengen agreement.

### Why is the United Kingdom not included in the European dataset despite having signed an agreement in association with the Schengen agreement?
Brexit. Despite the fact that the data has been gathered prior to the secession deadline, it would still be interesting to inquire on the UK data separately in case it would be of interest to compare changes in 2020 and onwards. And as of writing this, nothing has been officially said regarding future signed agreement in association with the Schengen agreement for UK.

### Why are the countries in APT / RCEP not included?
Unfortunately the data gathered for China, South Korea, and Japan are too small to make reliable inferences - and there are currently significant restrictions in freedom of movement for professionals between these countries, making it unreasonable to conduct an inquiry on these countries as a collective.

# 2. Data Preparation

In [24]:
del data['Respondent']

data = data[data["MainBranch"] == 'I am a developer by profession']
data = data[data["LastHireDate"] != "I've never had a job"]

len(data)

50397

In [0]:
relevant = ["Hobbyist", "OpenSourcer", "Employment", "Country",
"Student", "EdLevel", "UndergradMajor", "EduOther", "OrgSize",
"DevType", "YearsCode", "YearsCodePro", "JobSeek", "LastHireDate",
"FizzBuzz", "CompTotal", "CompFreq", "CurrencySymbol",
"LanguageWorkedWith", "DatabaseWorkedWith", "PlatformWorkedWith", "WebFrameWorkedWith", "MiscTechWorkedWith"]

data_us = data[data["Country"] == "United States"]
data_us = data_us[relevant]

data_uk = data[data["Country"] == "United Kingdom"]
data_uk = data_uk[relevant]

data_eu = data[data["Country"].isin(["France", "Germany", "Sweden",
                                     "Denmark", "Finland", "Norway",
                                     "Ireland", "Netherlands", "Austria",
                                     "Belgium", "Switzerland"])]
data_eu = data_eu[relevant]

data_test = data[data["Country"] == "Canada"]
data_test = data_test[relevant]

In [0]:
devtypes = parse_types(data.DevType)
eduother = parse_types(data.EduOther)
languages = parse_types(data.LanguageWorkedWith)
databases = parse_types(data.DatabaseWorkedWith)
platforms = parse_types(data.PlatformWorkedWith)
webframes = parse_types(data.WebFrameWorkedWith)
misctechs = parse_types(data.MiscTechWorkedWith)

In [30]:
print(devtypes)
print(eduother)
print(languages)
print(databases)
print(platforms)
print(webframes)
print(misctechs)

['Developer, full-stack', 'Database administrator', 'Developer, back-end', 'Developer, front-end', 'Developer, QA or test', 'DevOps specialist', 'Data or business analyst', 'Data scientist or machine learning specialist', 'Developer, desktop or enterprise applications', 'Developer, game or graphics', 'Educator', 'Developer, mobile', 'Engineer, data', 'Engineer, site reliability', 'Engineering manager', 'Senior executive/VP', 'System administrator', 'Student', 'Designer', 'Developer, embedded applications or devices', 'Product manager', 'Academic researcher', 'Scientist', 'Marketing or sales professional', 'nan']
['Taken an online course in programming or software development (e.g. a MOOC)', 'Received on-the-job training in software development', 'Taught yourself a new language, framework, or tool without taking a formal course', 'Participated in a full-time developer training program or bootcamp', 'Taken a part-time in-person course in programming or software development', 'Participate

In [0]:
data_us["CompTotal"] *= data_us["CompFreq"].apply(currency_annualize)
data_us["CompTotal"] *= data_us["CurrencySymbol"].apply(currency_usd)
data_us = data_us[data_us.CompTotal > 10000]
data_us = data_us[data_us.CompTotal < 1000000]

data_uk["CompTotal"] *= data_uk["CompFreq"].apply(currency_annualize)
data_uk["CompTotal"] *= data_uk["CurrencySymbol"].apply(currency_usd)
data_uk = data_uk[data_uk.CompTotal > 10000]
data_uk = data_uk[data_uk.CompTotal < 1000000]

data_eu["CompTotal"] *= data_eu["CompFreq"].apply(currency_annualize)
data_eu["CompTotal"] *= data_eu["CurrencySymbol"].apply(currency_usd)
data_eu = data_eu[data_eu.CompTotal > 10000]
data_eu = data_eu[data_eu.CompTotal > 1000000]

# 3. Data Exploration

In [0]:
for i in relevant:
    print(i + ": ")
    print(data[i].unique())
    
"""
Hobbyist: onehotencode
OpenSourcer: onehotencoding
Employment: onehotencoding
Country: onehotencoding
Student: onehotencoding
EdLevel: onehotencoding
UndergradMajor: onehotencoding
EduOther: multilabelencoder
OrgSize: onehotencoding
DevType: multilabelencoder
YearsCode: numerical
YearsCodePro: numerical
JobSeek: onehotencoding
LastHireDate: onehotencoding
FizzBuzz: onehotencode
CompTotal : y
Languages: multilabel
Database: multilabel
Platform: multilabel
Webframes: multilabel
MiscTech: multilabel
"""

In [37]:
data_dsml = []
for i in range( len(data_eu) ):
    if "Data scientist or machine learning specialist" in parse(data_eu.DevType.iloc[i]):
        data_dsml.append(data_eu.iloc[i])
        
pd.DataFrame(data_dsml)

,Hobbyist,OpenSourcer,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,YearsCodePro,JobSeek,LastHireDate,FizzBuzz,CompTotal,CompFreq,CurrencySymbol,LanguageWorkedWith,DatabaseWorkedWith,PlatformWorkedWith,WebFrameWorkedWith,MiscTechWorkedWith
1488,Yes,Once a month or more often,"Independent contractor, freelancer, or self-em...",Germany,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",NaN,"Just me - I am a freelancer, sole proprietor, ...",Academic researcher;Data or business analyst;D...,29,22,I am not interested in new job opportunities,NA - I am an independent contractor or self em...,No,3300000.0,Monthly,EUR,Assembly;Bash/Shell/PowerShell;C;C++;HTML/CSS;...,MariaDB;MongoDB;MySQL;PostgreSQL;Redis;SQLite,Arduino;Docker;Kubernetes;Linux;MacOS;Raspberr...,Angular/Angular.js;Express;jQuery;Laravel;Reac...,Node.js;Puppet;Unity 3D
2924,Yes,Less than once per year,Employed full-time,Norway,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",Taken an online course in programming or softw...,2-9 employees,Data scientist or machine learning specialist,28,12,"I’m not actively looking, but I am open to new...",1-2 years ago,No,1227600.0,Monthly,NOK,Python;SQL;VBA,MySQL;PostgreSQL,Slack,Flask,Pandas;TensorFlow
6318,Yes,Once a month or more often,"Independent contractor, freelancer, or self-em...",Germany,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",Taken an online course in programming or softw...,100 to 499 employees,Data scientist or machine learning specialist;...,34,26,"I’m not actively looking, but I am open to new...",NA - I am an independent contractor or self em...,No,1782000.0,Monthly,EUR,Java;JavaScript;Kotlin;Python;Ruby;Scala;Other...,Other(s):,AWS;Docker;Google Cloud Platform;Heroku;Kubern...,React.js;Spring;Other(s):,Apache Spark;Node.js;Pandas;Other(s):
18814,Yes,Once a month or more often,"Independent contractor, freelancer, or self-em...",Switzerland,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",Taken an online course in programming or softw...,"10,000 or more employees",Data or business analyst;Data scientist or mac...,17,17,"I’m not actively looking, but I am open to new...",1-2 years ago,No,3168000.0,Monthly,CHF,Bash/Shell/PowerShell;Dart;Go;HTML/CSS;Java;Ja...,Cassandra;Firebase;PostgreSQL;Redis,Android;Docker;Kubernetes;Linux;MacOS,Angular/Angular.js;Express;Flask;React.js,Flutter;Node.js;TensorFlow
20205,Yes,Once a month or more often,Employed full-time,Switzerland,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",Taken a part-time in-person course in programm...,2-9 employees,Academic researcher;Data scientist or machine ...,16,9,"I’m not actively looking, but I am open to new...",1-2 years ago,Yes,2400000.0,Monthly,USD,C#;Go;HTML/CSS;JavaScript;Python;SQL;TypeScript,DynamoDB;Elasticsearch;Redis,Android;AWS;Docker;Google Cloud Platform;IBM C...,ASP.NET;Express;React.js;Vue.js,.NET Core;Node.js;TensorFlow
23725,Yes,Less than once a month but more than once per ...,Employed part-time,Austria,"Yes, full-time","Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...","Taught yourself a new language, framework, or ...",2-9 employees,Data scientist or machine learning specialist;...,9,5,"I’m not actively looking, but I am open to new...",1-2 years ago,No,1003200.0,Monthly,EUR,Assembly;Bash/Shell/PowerShell;C;C++;C#;Dart;G...,Firebase;MariaDB;MongoDB;MySQL;Oracle;PostgreS...,Android;AWS;Docker;Google Cloud Platform;Kuber...,Angular/Angular.js;Django;Flask;jQuery;React.j...,Apache Spark;CryEngine;Node.js;Pandas;TensorFl...
26782,Yes,Less than once per year,Employed full-time,Germany,"Yes, part-time","Other doctoral degree (Ph.D, Ed.D., etc.)","Computer science, computer engineering, or sof...",

In [34]:
check_average_salary_column("DevType", data_us)        

#for i in parse_types(data_eu.DevType):
#    print(str(i) + ": " + str(check_average_salary(i, "DevType", data_eu)))

#check_average_salary("Data scientist or machine learning specialist", "DevType", data_eu)

Developer, full-stack: 125509.45467682606
Data or business analyst: 126699.353227771
Database administrator: 125529.55882352941
Developer, back-end: 132814.03635482307
Developer, desktop or enterprise applications: 127093.85968159941
Developer, front-end: 122616.6513064133
Developer, mobile: 126657.07311586052
DevOps specialist: 134982.06890130354
Engineer, data: 140940.1467611336
Engineer, site reliability: 149178.5287356322
Engineering manager: 163731.17370325694
Senior executive/VP: 176870.32930513597
System administrator: 126829.46859083192
Designer: 119892.48759007205
Developer, embedded applications or devices: 131939.20958646617
Developer, QA or test: 115920.95867026056
Educator: 137121.6194029851
Product manager: 133240.67342342343
Data scientist or machine learning specialist: 142127.42538190365
Developer, game or graphics: 122080.60897435897
Student: 107057.48796498906
Scientist: 140612.96927374302
Academic researcher: 139414.8161764706
nan: 141454.79166666666
Marketing or sa

In [0]:
data_dsml = data_eu["DevType" == ]

In [70]:
data_eu.DevType.iloc[5]

data_eu.CompTotal.iloc[5]

data_eu.Country.iloc[5]

parse(data_eu.DevType.iloc[5])

['Data or business analyst',
 'Data scientist or machine learning specialist',
 'Database administrator',
 'Designer',
 'Developer, back-end',
 'Developer, desktop or enterprise applications',
 'Developer, front-end',
 'Developer, full-stack',
 'Educator',
 'Marketing or sales professional',
 'Student',
 'System administrator']

In [0]:
#hob = encode(data["Hobbyist"])
#ope = encode(data["OpenSourcer"])

data_model = pd.concat([encode(data["Hobbyist"]),
                        encode(data["OpenSourcer"])
                       ,encode(data["Employment"])
                       ,encode(data["Country"])
                       ,encode(data["Student"])
                       ,encode(data["EdLevel"])
                       ,encode(data["UndergradMajor"])
                       ,encodem(data["EduOther"])
                       ,encode(data["OrgSize"])], axis = 1)
#encodem(data["DevType"])

In [46]:
data_model

,No,Yes,Less than once a month but more than once per year,Less than once per year,Never,Once a month or more often
1,1,0,0,0,1,0
3,0,1,0,0,0,1
4,0,1,0,0,0,1
5,0,1,1,0,0,0
6,0,1,0,1,0,0
7,0,1,0,0,1,0
8,0,1,1,0,0,0
10,0,1,0,0,1,0
12,0,1,0,1,0,0
13,0,1,0,1,0,0
